In [1]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *
PATH="/home/kirana/Documents/phd/final"
DATAPATH="/home/kirana/Documents/phd/data/aclImdb"

## Dataset

In [2]:
[df_train,df_valid,df_test,itos, train_tokens, valid_tokens, test_tokens, trn_lm, val_lm, test_lm]=pickle.load(open(f'{PATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [3]:
type(df_train)

pandas.core.frame.DataFrame

In [4]:
df_train=pd.concat([df_train,df_valid],axis=0)

In [5]:
df_train.shape

(75000, 4)

In [6]:
df_valid=df_test.copy()

In [7]:
df_train=df_train.loc[df_train['label']<2]

In [8]:
df_train.shape

(25000, 4)

In [9]:
df_train.head()

,label,text,words,tokens
3218,0,I've noticed how all the other reviews of this...,"[ \n , xxbos, xxfld, 1, i, 've, noticed, how, ...","[41, 42, 43, 40, 13, 161, 2030, 110, 44, 3, 10..."
6854,0,I went on a visit to one of my relatives a whi...,"[ \n , xxbos, xxfld, 1, i, went, on, a, visit,...","[41, 42, 43, 40, 13, 436, 30, 7, 2074, 9, 39, ..."
2697,0,Darcy and her young daughter Pamela are headin...,"[ \n , xxbos, xxfld, 1, xxmaj, darcy, and, her...","[41, 42, 43, 40, 2, 9225, 6, 55, 207, 551, 2, ..."
5747,0,Battlestar Gallactica was so great because it ...,"[ \n , xxbos, xxfld, 1, xxmaj, battlestar, xxm...","[41, 42, 43, 40, 2, 7900, 2, 33242, 20, 52, 10..."
8731,0,"Any movie with ""National Lampoon"" in the title...","[ \n , xxbos, xxfld, 1, xxmaj, any, movie, wit...","[41, 42, 43, 40, 2, 120, 25, 23, 16, 2, 2155, ..."


In [10]:
type(df_train['tokens'])

pandas.core.series.Series

In [11]:
bs=52 # 52 - Jeremey, 20 - default

In [12]:
bptt= 70 #70 - Jeremey, 35 - default

In [13]:
df_train.head()

,label,text,words,tokens
3218,0,I've noticed how all the other reviews of this...,"[ \n , xxbos, xxfld, 1, i, 've, noticed, how, ...","[41, 42, 43, 40, 13, 161, 2030, 110, 44, 3, 10..."
6854,0,I went on a visit to one of my relatives a whi...,"[ \n , xxbos, xxfld, 1, i, went, on, a, visit,...","[41, 42, 43, 40, 13, 436, 30, 7, 2074, 9, 39, ..."
2697,0,Darcy and her young daughter Pamela are headin...,"[ \n , xxbos, xxfld, 1, xxmaj, darcy, and, her...","[41, 42, 43, 40, 2, 9225, 6, 55, 207, 551, 2, ..."
5747,0,Battlestar Gallactica was so great because it ...,"[ \n , xxbos, xxfld, 1, xxmaj, battlestar, xxm...","[41, 42, 43, 40, 2, 7900, 2, 33242, 20, 52, 10..."
8731,0,"Any movie with ""National Lampoon"" in the title...","[ \n , xxbos, xxfld, 1, xxmaj, any, movie, wit...","[41, 42, 43, 40, 2, 120, 25, 23, 16, 2, 2155, ..."


In [14]:
df_train['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [15]:
df_valid['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [16]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', 'the', '.']

In [17]:
padlen=1400
padding_idx=1

In [18]:
df_train['n_tok']=df_train['tokens'].apply(len)

In [19]:
df_train['n_tok'].describe()

count    25000.000000
mean       304.945840
std        224.981807
min         16.000000
25%        166.000000
50%        228.000000
75%        371.000000
max       3354.000000
Name: n_tok, dtype: float64

In [20]:
df_train.sort_values(by='n_tok', inplace=True)

In [21]:
df_valid['n_tok']=df_valid['tokens'].apply(len)
df_valid.sort_values(by='n_tok', inplace=True)

In [22]:
df_test['n_tok']=df_test['tokens'].apply(len)
df_test.sort_values(by='n_tok', inplace=True)

In [23]:
for i in range(0,20):
    x=df_train['tokens'].values[i]
    y=df_train['label'].values[i]    
    out=np.ones(padlen)
    if len(x) < padlen:
        out[:len(x)]=x
    else:
        out=x[:padlen]
    print (out)
    print (y)
    print (len(out))
    if i>10:
        break

[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400


In [24]:
df_train.shape

(25000, 5)

In [25]:
df_train['label'].value_counts(), df_valid['label'].value_counts(),df_test['label'].value_counts()

(1    12500
 0    12500
 Name: label, dtype: int64, 1    12500
 0    12500
 Name: label, dtype: int64, 1    12500
 0    12500
 Name: label, dtype: int64)

In [26]:
df_train.shape, df_valid.shape

((25000, 5), (25000, 5))

In [27]:
padlen

1400

In [28]:
class ds_sentiment (Dataset):
    def __init__ (self,df,bs,padlen=64,xvar='tokens',yvar='label',len_var='n_tok',padding_idx=1):
        self.x,self.y,self.padlen,self.padding_idx,self.len_var,self.bs=\
            df[xvar],df[yvar],padlen,padding_idx,df[len_var],bs
        self.len_var=self.len_var.clip(0,padlen)
    
    def pad (self,x):
        out=np.ones(self.padlen)*self.padding_idx
        out=out.astype(int)
        if len(x)>=self.padlen:
            out[:]=x[:self.padlen]
        else:
            out[:len(x)]=x
        return out
        
    def __getitem__(self,idx):
        #import pdb
        #pdb.set_trace()
        return self.pad(self.x.iloc[idx]),self.y.iloc[idx],self.len_var.iloc[idx]
    
    def __len__(self):
        return len(self.x)        


In [29]:
#padlen=max(df_train['n_tok'])
padlen=1400

In [30]:
padlen

1400

In [31]:
dstrain=ds_sentiment(df_train,bs,padlen)
dsvalid=ds_sentiment(df_valid,bs,padlen)
dstest=ds_sentiment(df_test,bs,padlen)

In [32]:
dltrain=DataLoader(dstrain,bs,True)
dlvalid=DataLoader(dsvalid,bs,False)
dltest=DataLoader(dstest,bs,False)

In [33]:
for xb,yb,xlen in dltrain:
    break

In [34]:
xb, yb, xlen

(tensor([[41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         ...,
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1]]),
 tensor([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
         0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1,
         1, 0, 0, 1]),
 tensor([ 196,  525,   91,  167,  457,  450,  439,  413,  650,  602,  264,  202,
          138,  177,  515,  311,  182,  253,  490,  360,  153,  768,  186,  247,
          692,  168,  676,  142,  148,  121,  184,  210,  273,  193, 1316,  276,
          201,  388,   75,  657,  662,  281,  275,  400,  335,  268,  144,  130,
          277,  289,  193,  881]))

## Model Architecture

In [35]:
from torch import nn
import torch
from torch.autograd import Variable

In [36]:
bs,bptt

(52, 70)

In [37]:
n_inp=len(itos)
n_emb=400 #650
n_hidden=400#400
n_layers= 2 # 2
dropout=0.5 # 0.5
wd=1e-6
bidirectional=True
dropout_e=0.2 # 0.5
dropout_o=0.5 #0.5
n_out=1

In [38]:
xb.shape,xlen.shape

(torch.Size([52, 1400]), torch.Size([52]))

In [39]:
df_train['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [40]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [41]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', 'the', '.']

In [42]:
import torch

In [43]:
m = nn.AdaptiveAvgPool1d(5)
input = torch.randn(1, 64, 8)
input.size()

torch.Size([1, 64, 8])

In [44]:
m(input).size()

torch.Size([1, 64, 5])

In [71]:
class sentiment_classifier (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,n_out=1,padding_idx=1,n_filters=100,filter_sizes=[3,4,5]):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx,self.padding_idx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx,padding_idx
        self.n_out,self.n_filters,self.filter_sizes=n_out,n_filters,filter_sizes
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        
        self.create_architecture()
        if pretrain_mtx is not None:
            print (f'initializing glove with {pretrain_mtx.shape}')
            self.initialize_glove()
        self.init_hidden()
        self.criterion=nn.BCEWithLogitsLoss()
    
    def set_dropouts(self, dropout, dropout_o, dropout_e):
        self.dropout, self.dropout_o, self.dropout_e = dropout, dropout_o, dropout_e
    
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
         
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True

    def initialize_glove(self):
        self.encoder.weight.data.copy_(torch.Tensor(self.pretrain_mtx))
    
    def init_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    
      
    def create_architecture_cnn(self):
        # Dropout layer
        self.dropout_enc=nn.Dropout(self.dropout_e)
        # Embedding Layer: Embedding layer just maps each word to an index. n_inp to n_emb mapping is all it does
            # input to this is of shape n_batch * n_seq
        self.encoder=nn.Embedding(self.n_inp,self.n_emb,padding_idx=self.padding_idx)
          # embs are going to be of shape n_batch * n_seq * n_emb
            
            #embedding dimension is the "depth" of the filter and the number of tokens in the sentence is the width.
        self.conv_0=torch.nn.Conv1d (self.n_emb,self.n_filters,kernel_size=self.filter_sizes[0])
        self.conv_1=torch.nn.Conv1d (self.n_emb,self.n_filters,kernel_size=self.filter_sizes[1])
        self.conv_2=torch.nn.Conv1d(self.n_emb,self.n_filters,kernel_size=self.filter_sizes[2])

        # 
        
        
        
        self.fc=nn.Linear(len(self.filter_sizes)*self.n_filters,self.n_out)
        self.dropout_op=nn.Dropout(self.dropout_o)
                                    
    def create_architecture_lstm(self):
        # Dropout layer
        self.dropout_enc=nn.Dropout(self.dropout_e)
        # Embedding Layer: Embedding layer just maps each word to an index. n_inp to n_emb mapping is all it does
            # input to this is of shape n_batch * n_seq
        self.encoder=nn.Embedding(self.n_inp,self.n_emb,padding_idx=self.padding_idx)
             # embs are going to be of shape n_batch * n_seq * n_emb
            # h and c are of shape n_layers * n_batch * n_hidden
        # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=self.bidirectional)
        
                # output is of shape bs * n_seq * n_hidden
            # output [:,-1,:] is the same as hn[-1] but contains the padding idx also. Those are not there for hn[-1]
        self.dropout_op=nn.Dropout(self.dropout_o)
        
        self.avg_pool1d=torch.nn.AdaptiveAvgPool1d(1)
        self.max_pool1d=torch.nn.AdaptiveMaxPool1d(1)
        
        self.fc1=nn.Linear(self.n_hidden*4,self.n_hidden)
        self.fc2=nn.Linear(self.n_hidden*3,self.n_out)

        #self.log_softmax=nn.LogSoftmax()
        self.sigmoid=nn.Sigmoid()

    def create_architecture(self):
        ###################################
        # Embedding layer - common to both
        ###################################
        self.dropout_enc=nn.Dropout(self.dropout_e)
        self.encoder=nn.Embedding(self.n_inp,self.n_emb,padding_idx=self.padding_idx)
        
        #######################################
        # For RNN #############################
        #######################################
        # Embedding Layer: Embedding layer just maps each word to an index. n_inp to n_emb mapping is all it does
            # input to this is of shape n_batch * n_seq
         # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=self.bidirectional)
          # embs are going to be of shape n_batch * n_seq * n_emb
        self.dropout_op=nn.Dropout(self.dropout_o)
        
        self.avg_pool1d=torch.nn.AdaptiveAvgPool1d(1)
        self.max_pool1d=torch.nn.AdaptiveMaxPool1d(1)
        self.fc1=nn.Linear(self.n_hidden*2,self.n_hidden)
        self.fc2=nn.Linear(self.n_hidden*3,self.n_out)

        #self.log_softmax=nn.LogSoftmax()
        self.sigmoid=nn.Sigmoid()
    
        #######################################
        # For CNN #############################
        #######################################    
        #embedding dimension is the "depth" of the filter and the number of tokens in the sentence is the width.
        self.conv_0=torch.nn.Conv1d (self.n_emb,self.n_filters,kernel_size=self.filter_sizes[0])
        self.conv_1=torch.nn.Conv1d (self.n_emb,self.n_filters,kernel_size=self.filter_sizes[1])
        self.conv_2=torch.nn.Conv1d(self.n_emb,self.n_filters,kernel_size=self.filter_sizes[2])
        self.fc=nn.Linear(len(self.filter_sizes)*self.n_filters,self.n_out)
        self.dropout_op=nn.Dropout(self.dropout_o)
        # 
        
        ###################################
        ## MERGE THE BOTH OF THEM
        
        self.logisticreg=nn.Linear(2,1)
        
    def forward (self,Xb,Yb,Xb_lengths):
        
        ####RNN PORTION
        embs=self.dropout_enc(self.encoder(Xb))
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())
        packed_embs = pack_padded_sequence(embs,Xb_lengths,batch_first=True, enforce_sorted=False)
        lstm_out,(hidden,cell)=self.lstm(packed_embs)
        lstm_out,lengths=pad_packed_sequence(lstm_out,batch_first=True)
        hidden = self.dropout_op(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        avg_pool=self.avg_pool1d(lstm_out.permute(0,2,1)).view(Xb.size(0),-1)
        max_pool=self.max_pool1d(lstm_out.permute(0,2,1)).view(Xb.size(0),-1)
        #max_avg=torch.cat([avg_pool,max_pool],dim=1)
        preds_max_avg=self.fc1(max_pool)
        big_out=torch.cat([hidden,preds_max_avg],dim=1)
        preds_rnn=self.fc2(big_out)
        
        #CNN Portion
        new_embs=embs.permute(0,2,1)
        
        conved_0=torch.relu(self.conv_0(new_embs))
        conved_1=torch.relu(self.conv_1(new_embs))
        conved_2=torch.relu(self.conv_2(new_embs))
        
        
        max_pool1d=torch.nn.MaxPool1d(conved_0.shape[2])
        pooled_0=max_pool1d(conved_0).squeeze(2)
        max_pool1d=torch.nn.MaxPool1d(conved_1.shape[2])
        pooled_1=max_pool1d(conved_1).squeeze(2)
        max_pool1d=torch.nn.MaxPool1d(conved_2.shape[2])
        pooled_2=max_pool1d(conved_2).squeeze(2)

        cat_cnn = self.dropout_op(torch.cat([pooled_0,pooled_1,pooled_2],dim=1))
        preds_cnn= self.fc (cat_cnn)
       
        
        loss_rnn=self.criterion(preds_cnn,Yb.contiguous().float().view(-1,1)) 
        loss_cnn=self.criterion(preds_rnn,Yb.contiguous().float().view(-1,1))
        #return preds_cnn.view(-1), preds_rnn.view(-1),loss_cnn,loss_rnn 
        if 1==0:
            preds=self.logisticreg(torch.cat([preds_rnn,preds_cnn],1))
            loss=self.criterion(preds,Yb.contiguous().float().view(-1,1))
        loss=0.7*loss_rnn+0.3*loss_cnn
        preds=(preds_cnn*0.3+preds_rnn*0.7)
        return preds.view(-1),loss
        
    def forward_cnn(self,Xb,Yb,Xb_lengths):
        embs=self.dropout_enc(self.encoder(Xb))

        embs=embs.permute(0,2,1)
        
        conved_0=torch.relu(self.conv_0(embs))
        conved_1=torch.relu(self.conv_1(embs))
        conved_2=torch.relu(self.conv_2(embs))
        
        
        max_pool1d=torch.nn.MaxPool1d(conved_0.shape[2])
        pooled_0=max_pool1d(conved_0).squeeze(2)
        max_pool1d=torch.nn.MaxPool1d(conved_1.shape[2])
        pooled_1=max_pool1d(conved_1).squeeze(2)
        max_pool1d=torch.nn.MaxPool1d(conved_2.shape[2])
        pooled_2=max_pool1d(conved_2).squeeze(2)

        cat_cnn = self.dropout_op(torch.cat([pooled_0,pooled_1,pooled_2],dim=1))
        preds= self.fc (cat_cnn)
       
              
        
        loss=self.criterion(preds,Yb.contiguous().float().view(-1,1))
        
        return preds.view(-1), loss                               
                                    
  
                                    
                                    
    def forward_rnn(self,Xb,Yb,Xb_lengths):
        embs=self.dropout_enc(self.encoder(Xb))
        
        
        
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())

        # before lstm call
            # pack_padded_sequence so that padded items in the sequence won't be shown to the LSTM
        packed_embs = pack_padded_sequence(embs,Xb_lengths,batch_first=True, enforce_sorted=False)
        lstm_out,(hidden,cell)=self.lstm(packed_embs)

        # after lstm call
        lstm_out,lengths=pad_packed_sequence(lstm_out,batch_first=True)
            # lstm_out is of shape bs * n_seq * (n_hidden*n_layers)
        
        # lstm_out has the values that are padded
        hidden = self.dropout_op(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        #import pdb
        #pdb.set_trace()
        avg_pool=self.avg_pool1d(lstm_out.permute(0,2,1)).view(Xb.size(0),-1)
        max_pool=self.max_pool1d(lstm_out.permute(0,2,1)).view(Xb.size(0),-1)
        
        # concatenate
        max_avg=torch.cat([avg_pool,max_pool],dim=1)
        preds_max_avg=self.fc1(max_avg)
        
        ## Concatenate hidden, average pool and max pool

        big_out=torch.cat([hidden,preds_max_avg],dim=1)
        preds=self.fc2(big_out)
        
              
        
        loss=self.criterion(preds,Yb.contiguous().float().view(-1,1))
        
        return preds.view(-1), loss  

In [72]:
def accuracy_binomial(preds,actual, device="cpu", cutoff=0.5):
    preds=torch.sigmoid(preds)
    zeros=torch.zeros(len(preds)).to(device)
    ones = torch.ones(len(preds)).to(device)

    preds=torch.where(preds>cutoff,ones,zeros)
    correct=torch.round(preds).long()==actual
    return correct.float().sum()/len(correct)

In [73]:
def accuracy_binomial(preds, y, device="cpu", cutoff=0.5):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    y=y.float()
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [74]:
device="cpu"

## Load pretrained model weights

In [75]:
pretrained_lm_weights=pickle.load(open(f'{PATH}/inter/pretrained_lm_weights','rb'))

In [76]:
if 1==0:
    model=torch.load (f'{PATH}/inter/model_awd_lstm')
    pretrained_lm_weights=model.encoder.weight.data.cpu().numpy()
    import pickle
    pickle.dump(pretrained_lm_weights,open(f'{PATH}/inter/pretrained_lm_weights','wb'))

### Check if model forward works

In [77]:
n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout_o,dropout,n_out

(59972, 400, 400, 2, True, 52, 'cpu', 0.2, 0.5, 0.5, 1)

In [78]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=dropout_e,dropout=dropout,\
                 dropout_o=dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)


initializing glove with (59972, 400)


In [79]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [80]:
print(f'The model has {count_parameters(model_sentiment):,} trainable parameters')

The model has 31,203,805 trainable parameters


In [81]:
model_sentiment

sentiment_classifier(
  (dropout_enc): Dropout(p=0.2)
  (encoder): Embedding(59972, 400, padding_idx=1)
  (lstm): LSTM(400, 400, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout_op): Dropout(p=0.5)
  (avg_pool1d): AdaptiveAvgPool1d(output_size=1)
  (max_pool1d): AdaptiveMaxPool1d(output_size=1)
  (fc1): Linear(in_features=800, out_features=400, bias=True)
  (fc2): Linear(in_features=1200, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (conv_0): Conv1d(400, 100, kernel_size=(3,), stride=(1,))
  (conv_1): Conv1d(400, 100, kernel_size=(4,), stride=(1,))
  (conv_2): Conv1d(400, 100, kernel_size=(5,), stride=(1,))
  (fc): Linear(in_features=300, out_features=1, bias=True)
  (logisticreg): Linear(in_features=2, out_features=1, bias=True)
  (criterion): BCEWithLogitsLoss()
)

In [82]:
model_sentiment.forward(xb,yb,xlen)

(tensor([ 0.2781,  0.0711,  0.1606,  0.2173,  0.1255, -0.0008,  0.0793,  0.1880,
          0.0227,  0.1950,  0.0488,  0.1719,  0.1151,  0.1110,  0.2271,  0.1619,
          0.0398,  0.0323,  0.2833,  0.1850,  0.2999,  0.1686, -0.1080,  0.0611,
          0.2400,  0.2493,  0.0953,  0.1903,  0.0915,  0.1094,  0.0643,  0.3108,
          0.0657, -0.0063,  0.2532,  0.0899,  0.0776,  0.0246,  0.2288, -0.0328,
          0.2514,  0.1069,  0.0803,  0.1808, -0.0217,  0.1983,  0.1610, -0.0883,
         -0.0733,  0.1578,  0.1862,  0.0739], grad_fn=<ViewBackward>),
 tensor(0.7219, grad_fn=<AddBackward0>))

In [83]:
preds,loss=model_sentiment.forward(xb,yb,xlen)

In [84]:
preds.view(-1)

tensor([ 0.0284,  0.2606,  0.1308,  0.1175,  0.2756,  0.3299,  0.0917,  0.5204,
         0.2249,  0.3595,  0.2504,  0.2372,  0.1425,  0.4310,  0.2673,  0.1575,
         0.0597,  0.2745,  0.0218,  0.0675,  0.3260, -0.0492,  0.0571,  0.2034,
         0.0736,  0.0941,  0.1982,  0.2905, -0.0401,  0.2669,  0.1736,  0.0306,
        -0.1665,  0.0372,  0.0891,  0.3457,  0.2484,  0.1920, -0.1057,  0.1314,
         0.3000,  0.1200,  0.1997,  0.2153,  0.1872, -0.0255,  0.2300,  0.3364,
        -0.0838,  0.1835,  0.0799, -0.0445], grad_fn=<ViewBackward>)

In [85]:
yb

tensor([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1,
        1, 0, 0, 1])

In [86]:
preds.size()

torch.Size([52])

In [87]:
yb.size()

torch.Size([52])

In [88]:
accuracy_binomial(preds.to(device),yb.to(device))

tensor(0.3846)

In [89]:
from sklearn.metrics import roc_auc_score

In [90]:
roc_auc_score(yb,preds)

0.4029629629629629

In [91]:
pretrained_lm_weights

array([[-9.038397e-02, -1.795423e-02, -3.457930e-02,  8.671192e-02, ..., -3.003510e-02, -1.899482e-02,  1.426260e-02,
        -4.948893e-02],
       [-2.453323e-02, -3.018817e-02,  6.663179e-02, -5.793565e-02, ...,  1.840935e-02,  6.371137e-02,  9.835491e-03,
        -2.131385e-03],
       [ 7.696263e-03, -4.266643e-02, -1.533517e-01,  2.239776e-01, ...,  9.869517e-02,  3.041433e-02,  1.824751e-01,
         1.134978e-01],
       [ 3.305928e-02,  2.266591e-01, -4.264669e-02,  1.490862e-01, ...,  3.407921e-02, -6.422209e-03,  3.180612e-01,
         9.373549e-02],
       ...,
       [ 6.027538e-02, -5.980809e-02,  1.861691e-01, -3.105092e-02, ..., -2.764457e-02,  1.962678e-02, -2.172215e-03,
         6.297247e-02],
       [-1.592789e-02, -2.781571e-04,  1.301994e-01,  2.851282e-02, ...,  5.064877e-02,  1.670864e-01,  2.283701e-02,
        -8.746398e-03],
       [ 4.669700e-02,  3.138980e-02,  1.221957e-02, -3.927753e-02, ..., -1.281436e-01,  1.121320e-01, -3.726090e-03,
        -3.777364e

## Learner

In [92]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1
  
        
    
    def fit (self,Xb,Yb,Xlen,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb,Xlen)
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1),self.device)
            acc=acc.item()
            del preds
        
        if mode_train:
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.clip_val)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        epoch_loss,epoch_acc,i,k=0,0,0,0
        self.model.init_hidden()
        for Xb,Yb,Xlen in iterator:
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            Xlen=Xlen.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,Xlen,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            
            k=k+1
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
        epoch_loss=epoch_loss/len(iterator)
        epoch_acc=epoch_acc/len(iterator)
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

    
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        for epoch in range(n_epochs):
            
            loss,acc=self.run_epoch(dltrain,True)
            print (f'Epoch:{epoch} Loss:{loss}')
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Loss:{loss} Accuracy:{acc} Loss:{lossv} Accuracy:{accv}')
        

In [93]:
device="cuda:1"

In [94]:
n_layers

2

In [95]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout,\
                 dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)
model_sentiment=model_sentiment.to(device)

initializing glove with (59972, 400)


In [96]:
optimizer=torch.optim.Adam(model_sentiment.parameters(),lr=5e-3,betas=(0.9,0.999), weight_decay=wd)
metric_fn=accuracy_binomial

In [97]:
len(dltrain),len(dlvalid)

(481, 481)

In [98]:
df_valid['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [99]:
learner=Learner(model_sentiment,optimizer,accuracy_binomial,device,bptt,50,0.25)

In [100]:
model_sentiment.freeze_embedding()

In [101]:
model_sentiment.encoder.weight.requires_grad, learner.model.encoder.weight.requires_grad

(False, False)

In [102]:
model_sentiment.dropout_e,model_sentiment.dropout,model_sentiment.dropout_o, learner.model.dropout_e,learner.model.dropout,learner.model.dropout_o

(0.2, 0.5, 0.5, 0.2, 0.5, 0.5)

In [103]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:50 0.7584159255027771  0.5503846353292465
Batch:100 0.6645771166682244  0.6442307916283607
Batch:150 0.5943375905354817  0.6951282292604446
Batch:200 0.5372937221825123  0.7383654107153416
Batch:250 0.5016948594450951  0.7672307969331741
Batch:300 0.475238998234272  0.7867307983835539
Batch:350 0.45669078354324616  0.8017033267872674
Batch:400 0.4405833813548088  0.8130769541114569
Batch:450 0.4302807691362169  0.8211538776424196
Epoch:0 Loss:0.42242208860520264
Batch:50 0.25385377258062364  0.9188461935520172
Batch:100 0.25152337178587914  0.9169231188297272
Batch:150 0.24863764653603235  0.9176923477649689
Batch:200 0.24988415464758873  0.9160577324032784
Batch:250 0.2539145795106888  0.9135385010242462
Batch:300 0.2592463783919811  0.910833372871081
Batch:350 0.26158339010817666  0.9098901494911739
Batch:400 0.2639759146794677  0.9091827315092087
Batch:450 0.2669048407342699  0.907863286336263
Epoch:0 Loss:0.42242208860520264 Accuracy:0.8265832717725988 Loss:0.2667261921876185

In [104]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:50 0.2602949035167694  0.9246154272556305
Batch:100 0.26078327000141144  0.9253846567869186
Batch:150 0.26985132664442063  0.9215385015805563
Batch:200 0.2697112593054771  0.9200000390410423
Batch:250 0.2718077033162117  0.9196923470497131
Batch:300 0.27274388055006665  0.919615423878034
Batch:350 0.27353397182055883  0.9201099286760602
Batch:400 0.27226867550984024  0.9200481155514717
Batch:450 0.27181959892312685  0.9212393550078074
Epoch:0 Loss:0.2721488145402712
Batch:50 0.23855660259723663  0.9215384984016418
Batch:100 0.23052386477589606  0.9250000357627869
Batch:150 0.22719828700025876  0.927564138174057
Batch:200 0.22540926929563285  0.9287500357627869
Batch:250 0.22739271780848502  0.9279231135845184
Batch:300 0.22894382283091544  0.9265384991963704
Batch:350 0.23035185047558376  0.9248901471069881
Batch:400 0.23013888780027628  0.925000037252903
Batch:450 0.23098011559910245  0.9244444823265076
Epoch:0 Loss:0.2721488145402712 Accuracy:0.9211778733685706 Loss:0.230335042

In [105]:
for param_group in learner.optimizer.param_groups:
    param_group['lr']=5e-4

In [106]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:50 0.21061992287635803  0.9423077285289765
Batch:100 0.21618611991405487  0.9403846502304077
Batch:150 0.21317385693391164  0.9421795201301575
Batch:200 0.2095358297601342  0.9436538797616959
Batch:250 0.21004493841528893  0.9423846492767334
Batch:300 0.20874958125253518  0.9437820851802826
Batch:350 0.2082487084184374  0.9439560789721353
Batch:400 0.20598521264269948  0.945048111975193
Batch:450 0.20462557028565143  0.9453846507602268
Epoch:0 Loss:0.20349373961963427
Batch:50 0.2257000485062599  0.9257692682743073
Batch:100 0.21463873624801635  0.9275000381469727
Batch:150 0.20880756457646688  0.9305128578344981
Batch:200 0.2073708800598979  0.9311538836359978
Batch:250 0.2092157429754734  0.929769268989563
Batch:300 0.21088908096154532  0.9280128586292267
Batch:350 0.21207390189170838  0.9271978402137756
Batch:400 0.21261372368782758  0.9275481149554252
Batch:450 0.21493610123793283  0.9268803803126017
Epoch:0 Loss:0.20349373961963427 Accuracy:0.9456821079065795 Loss:0.21543617

In [107]:
for param_group in learner.optimizer.param_groups:
    param_group['lr']=5e-5

In [108]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:50 0.18099605277180672  0.9553846514225006
Batch:100 0.17272445492446423  0.9578846514225006
Batch:150 0.17237537413835524  0.9570513180891673
Batch:200 0.17455839972943069  0.9554808053374291
Batch:250 0.1772473058998585  0.9539231126308441
Batch:300 0.1783899956693252  0.9541667022307714
Batch:350 0.17966196766921452  0.953736299276352
Batch:400 0.17963904932141303  0.9535577276349068
Batch:450 0.17836755072077115  0.9536752484904395
Epoch:0 Loss:0.1775560694090294
Batch:50 0.22347456246614456  0.9242308032512665
Batch:100 0.21099014237523078  0.9273077285289765
Batch:150 0.20516699055830637  0.930897472302119
Batch:200 0.2047584541887045  0.930480805337429
Batch:250 0.20696539029479027  0.9300000369548798
Batch:300 0.2084414041787386  0.9294231142600378
Batch:350 0.2091118818947247  0.9287363009793418
Batch:400 0.20920589435845613  0.9291827297210693
Batch:450 0.21117532258232435  0.9285897809929318
Epoch:0 Loss:0.1775560694090294 Accuracy:0.9541980194202827 Loss:0.21064545959

In [109]:
torch.save(model_sentiment.state_dict(),f'{PATH}/inter/sentiment_combmodel_nounfreeze_state_dict_0.928')
torch.save(optimizer.state_dict(),f'{PATH}/inter/sentiment_comblearner_nounfreeze_state_dict_0.928')
torch.save (model_sentiment,f'{PATH}/inter/sentiment_combmodel_nounfreeze_0.928')
torch.save (optimizer,f'{PATH}/inter/sentiment_comboptimizer_nounfreeze_0.928')
torch.save (learner,f'{PATH}/inter/sentiment_comblearner_nounfreeze_0.928')

/home/kirana/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type sentiment_classifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [110]:
model_sentiment.unfreeze_embedding()

In [111]:
model_sentiment.encoder.weight.requires_grad, learner.model.encoder.weight.requires_grad

(True, True)

In [112]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.17493538081645965  0.9561538791656494
Batch:100 0.17721302550286055  0.9548077237606049
Batch:150 0.17729540345569453  0.9569231100877126
Batch:200 0.18007498333230615  0.9548077258467674
Batch:250 0.17792524062097073  0.95584618806839
Batch:300 0.17829006606092054  0.9555128544569016
Batch:350 0.17789884659860816  0.9559890450750078
Batch:400 0.17710435344837605  0.9557211877405644
Batch:450 0.17813795316550468  0.9558547343148126
Epoch:0 Loss:0.178482661670434
Batch:50 0.2226167418062687  0.9253846490383149
Batch:100 0.21022986188530923  0.9280769598484039
Batch:150 0.20406683415174484  0.9311538835366567
Batch:200 0.20320395998656748  0.930865421295166
Batch:250 0.20514419555664062  0.9298461916446685
Batch:300 0.20654248200356962  0.9291026020050048
Batch:350 0.20751422758613314  0.9284615761893136
Batch:400 0.20778242031112312  0.9289904221892357
Batch:450 0.20982487615611817  0.9285043115086026
Epoch:0 Loss:0.178482661670434 Accuracy:0.9559691684657472 Loss:0.209823180

In [113]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:50 0.1796993599832058  0.9576923358440399
Batch:100 0.17213856488466261  0.9584615689516067
Batch:150 0.16667891293764114  0.9589743900299073
Batch:200 0.16918746769428253  0.9588461855053901
Batch:250 0.1691615637242794  0.9592308008670807
Batch:300 0.169596492399772  0.9590384934345881
Batch:350 0.16886687725782396  0.9589560759067536
Batch:400 0.17080709227360785  0.9581250323355198
Batch:450 0.16847730916407372  0.9587607159879472
Epoch:0 Loss:0.16862219942272824
Batch:50 0.22200749382376672  0.9234615743160248
Batch:100 0.2091979483515024  0.9265385007858277
Batch:150 0.20304773584008218  0.9300000381469726
Batch:200 0.20253973938524722  0.9300961908698082
Batch:250 0.2047864469587803  0.9294615759849548
Batch:300 0.20614869686464468  0.9289743967851003
Batch:350 0.2068827559053898  0.9282967409065791
Batch:400 0.20708336601033805  0.9288461914658547
Batch:450 0.20918306630518702  0.9283761061562432
Epoch:0 Loss:0.16862219942272824 Accuracy:0.9590476893833422 Loss:0.20871845

In [114]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.14831322476267814  0.9669231069087982
Batch:100 0.15890183448791503  0.9651923382282257
Batch:150 0.15571161108712356  0.966153877178828
Batch:200 0.15806500744074584  0.9662500303983689
Batch:250 0.1604635378420353  0.9652307999134063
Batch:300 0.16394074910630782  0.9637820820013682
Batch:350 0.16396197034844331  0.9634615690367563
Batch:400 0.16163033758290113  0.9640384916961193
Batch:450 0.16222475753062301  0.9635043039586809
Epoch:0 Loss:0.16214304859604756
Batch:50 0.22104861378669738  0.923076958656311
Batch:100 0.20808142654597758  0.9263461923599243
Batch:150 0.20183727160096168  0.9297436273097992
Batch:200 0.20117236562073232  0.9301923441886902
Batch:250 0.20324968230724336  0.9299231135845184
Batch:300 0.20449111166099707  0.9294872166713078
Batch:350 0.2054740560054779  0.9289011357511793
Batch:400 0.2057606272958219  0.929423113912344
Batch:450 0.2078460860417949  0.9288034560945299
Epoch:0 Loss:0.16214304859604756 Accuracy:0.963365615033806 Loss:0.207645952

In [115]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.16885913476347925  0.9615384936332703
Batch:100 0.15936354756355287  0.9651923406124115
Batch:150 0.1592694428563118  0.9644872109095256
Batch:200 0.15632881922647357  0.9650000315904618
Batch:250 0.155020054936409  0.9660000314712525
Batch:300 0.1532587735603253  0.9660897747675578
Batch:350 0.15422616588217872  0.9660989323684147
Batch:400 0.15426115840673446  0.9659615701436997
Batch:450 0.1547477538469765  0.9656410574913025
Epoch:0 Loss:0.15475054663797674
Batch:50 0.22095745116472243  0.9238461887836457
Batch:100 0.20784359097480773  0.9267308074235916
Batch:150 0.20166147058208783  0.9303846526145935
Batch:200 0.201059164442122  0.9307692670822143
Batch:250 0.20317641404271125  0.9301538825035095
Batch:300 0.20448637386163077  0.9293590112527211
Batch:350 0.205431331630264  0.9285714653560093
Batch:400 0.20577694775536656  0.9293269602954388
Batch:450 0.2079033212032583  0.9288034560945299
Epoch:0 Loss:0.15475054663797674 Accuracy:0.9655805533740228 Loss:0.20764656368

In [179]:
len(dltrain)

433